# Obermeyer Case Simulation

This notebook uses the 

In [108]:
import cupy as np

In [109]:
products = {
    "gail":(1017, 388),
    "isis":(1042, 646),
    "entice":(1358, 496)
}

In [4]:
product_distribution_moments = np.array(list(products.values()))
means = product_distribution_moments[:,0]
std_devs = product_distribution_moments[:,1]
production_capacity = np.sum(means)

In [34]:
def run_simul(first_order, num_simulations):
    # Generate the final demand
    gaussian_realizations = np.random.normal(loc=means, scale=std_devs, size=(num_simulations,3))
    gaussian_realizations[gaussian_realizations<0] = 0

    # Compute delta between demand and november order
    delta_orders = gaussian_realizations - first_order

    # Overstocks are products for which we already overshot during the first order
    overstock = np.absolute(np.sum(delta_orders * (delta_orders < 0), axis=1))
    # Missed sales is the products we won't be able to produce due to capacity
    # constraints despite knowing the end of season values
    remaining_demand = np.sum(delta_orders * (delta_orders > 0), axis=1)
    remaining_capacity = production_capacity - first_order.sum()
    missed_sales = remaining_demand - remaining_capacity
    missed_sales[missed_sales < 0] = 0

    # Costs on overstack are one per unit, missed sales are three per unit
    overstock_cost = overstock
    print(overstock_cost.mean())
    missed_sales_cost = missed_sales * 3
    print(missed_sales_cost.mean())
    total_cost = overstock_cost + missed_sales_cost
    
    return total_cost.mean()

In [104]:
half_mean_first_order = means/2
print('half mean order: {}, total: {}'.format(half_mean_first_order, half_mean_first_order.sum()))
overorder_quantity = 10
overorder_volatile_first_order = np.array([505.5-overorder_quantity/2, 527+overorder_quantity, 676-overorder_quantity/2])
print('over-ordering volatile styles: {}, total: {}'.format(overorder_volatile_first_order, overorder_volatile_first_order.sum()))
underorder_quantity = 100
underorder_volatile_first_order = np.array([529.5+underorder_quantity, 479-underorder_quantity*2, 700+underorder_quantity])
print('under-ordering volatile styles: {}, total: {}'.format(underorder_volatile_first_order, underorder_volatile_first_order.sum()))
num_simuls = int(1.5e8)

half mean order: [508.5 521.  679. ], total: 1708.5
over-ordering volatile styles: [500.5 537.  671. ], total: 1708.5
under-ordering volatile styles: [629.5 279.  800. ], total: 1708.5


In [105]:
products

{'gail': (1017, 388), 'isis': (1042, 646), 'entice': (1358, 496)}

In [106]:
def compare():
    half_mean_cost = run_simul(half_mean_first_order, num_simuls)
    over_volatile_cost = run_simul(overorder_volatile_first_order, num_simuls)
    under_volatile_cost = run_simul(underorder_volatile_first_order, num_simuls)
    print(half_mean_cost, over_volatile_cost, under_volatile_cost)

In [107]:
compare()

97.9497963809516
1123.686793802585
99.94302863456052
1124.768317833354
86.88655126037384
1126.6085605240175
1221.6365901835366 1224.7113464679146 1213.4951117843914


In [95]:
compare()

97.95374732674452
1123.5880728005193
201.28799553207938
1233.354941328994
92.29170463051692
1137.4481196434367
1221.541820127264 1434.6429368610738 1229.7398242739534


In [40]:
compare()

97.9419666540934
1123.6017674219443
98.71542727748297
1123.8240793558289
97.25509490876757
1123.398917380518
1221.5437340760375 1222.5395066333117 1220.6540122892854


In [41]:
compare()

97.94632870600378
1123.6983377492606
98.71154841047016
1124.1479305581495
97.20107164981046
1123.515711851174
1221.6446664552648 1222.8594789686194 1220.7167835009843


In [42]:
compare()

97.97328806239004
1123.771404679598
98.7104552737207
1124.0334142195268
97.24042685151024
1123.4787019067003
1221.744692741988 1222.7438694932473 1220.7191287582104


In [43]:
compare()

97.95116272387557
1123.714080565611
98.66843234772483
1124.0854341798133
97.24610881110931
1123.5188527575067
1221.6652432894866 1222.7538665275383 1220.764961568616


In [44]:
compare()

97.95008771622751
1123.669677454427
98.65317482518633
1124.1045224639029
97.24474261984815
1123.206198507281
1221.619765170655 1222.7576972890893 1220.450941127129


In [35]:
compare()

1221.7148905538807 1222.454892348385 1220.8409365982295


In [36]:
compare()

1221.5845082764438 1224.4660316531915 1222.6195832680896


In [37]:
compare()

1220.5282274385133 1223.406889070468 1219.0460015574893


In [38]:
compare()

1220.778882161316 1222.8744422023049 1222.0186713075282


In [39]:
compare()

1220.2606628633018 1223.6184939674383 1222.0911350260415


In [40]:
compare()

1222.6129590036721 1222.0321629457221 1221.09557416603


In [41]:
compare()

1220.866016866949 1222.3423217199183 1220.2260768745186


In [42]:
compare()

1221.7064263992443 1221.6180660337445 1221.6788947517857


In [43]:
compare()

1222.971979713675 1222.2443512384634 1220.956536866181


In [44]:
compare()

1222.005196493903 1223.829108377333 1219.0245363656525


In [45]:
compare()

1221.8113155688743 1221.7125237177265 1219.5097097787047


In [46]:
compare()

1222.1598055392378 1222.9979148299299 1219.7446950766127


In [47]:
compare()

1224.3856192965832 1222.8343057822876 1221.186819847121
